# Análisis del sentimiento en IMDB

Los datos se dividen en partes iguales: 25.000 reseñas para el entrenamiento y 25.000 para probar el clasificador. Además, cada conjunto tiene 12,5k críticas positivas y 12,5k negativas.

IMDb permite a los usuarios puntuar las películas en una escala del 1 al 10. Para etiquetar estas reseñas, el encargado de los datos etiquetó todo lo que tuviera ≤ 4 estrellas como negativo y todo lo que tuviera ≥ 7 estrellas como positivo. Las reseñas con 5 o 6 estrellas se omitieron.

**Importar las bibliotecas necesarias**

In [46]:
import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")

**Load Data**

In [47]:
reviews_train = []
for line in open(os.getcwd() + '/data/imbd_train.txt', 'r', encoding='latin1'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open(os.getcwd() + '/data/imbd_test.txt', 'r', encoding='latin1'):
    
    reviews_test.append(line.strip())

In [48]:
for i in range(3):
    print('####################')
    print(reviews_train[i])

####################
Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
####################
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once conside

**Ver uno de los elementos de la lista**

In [49]:
print(len(reviews_train))
print(len(reviews_test))
reviews_train[5]

25000
25000


"This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br />

El texto en bruto es bastante desordenado para hacer una revisión de esta manera, por lo que antes de que podamos hacer cualquier análisis tenemos que limpiar las cosas


**Utilizar expresiones regulares para eliminar los caracteres no textuales y las etiquetas html**.

In [50]:
import re

# Remover todos los signos de puntuación, exclamaciones...
# Tb pasamos a minuscula y nos cargamos etiquetas HTML
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    # Para todas las reviews en minuscula, sustituye algunas cosas por espacio y otras por vacio.
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

# Reviews tras aplicar la limpieza
reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [51]:
reviews_train_clean[5]

"this isn't the comedic robin williams nor is it the quirky insane robin williams of recent thriller fame this is a hybrid of the classic drama without over dramatization mixed with robin's new love of the thriller but this isn't a thriller per se this is more a mystery suspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until william's character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all it's worth a watch though it's definitely not friday saturday night fare it rates a   from the fiend "

# Vectorization
Para que estos datos tengan sentido para nuestro algoritmo de aprendizaje automático, tendremos que convertir cada opinión en una representación numérica, lo que llamamos vectorización.

La forma más sencilla de hacerlo es crear una matriz muy grande con una columna para cada palabra única del corpus (en nuestro caso, el corpus son las 50.000 reseñas). A continuación, transformamos cada opinión en una fila que contiene 0 y 1, donde 1 significa que la palabra del corpus correspondiente a esa columna aparece en esa opinión. Dicho esto, cada fila de la matriz será muy escasa (mayoritariamente ceros). Este proceso también se conoce como codificación en caliente. Utilice el método *CountVectorizer*.

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

In [53]:
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
]

In [54]:
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [55]:
len(vectorizer.get_feature_names_out())

9

In [56]:
print(X.toarray())

[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [57]:
# Si aparece una palabra en una review, le pone un 1. Da igual que aparezca 100 veces, no cuenta. Xq binary=True
# Solo pone 1s cuando detecta una palabra en una review
baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(reviews_train_clean)

# Reviews en formato vector de palabras. El mismo vectorizador a test, tiene que mantener la estructura
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [58]:
X_baseline

<25000x87063 sparse matrix of type '<class 'numpy.int64'>'
	with 3410713 stored elements in Compressed Sparse Row format>

In [59]:
print(X_baseline.shape)

# Asigna un numero según el orden de aparicion
baseline_vectorizer.vocabulary_

(25000, 87063)


{'bromwell': 9819,
 'high': 35211,
 'is': 39472,
 'cartoon': 11686,
 'comedy': 14754,
 'it': 39642,
 'ran': 61772,
 'at': 4537,
 'the': 76725,
 'same': 66138,
 'time': 77626,
 'as': 4211,
 'some': 71188,
 'other': 54861,
 'programs': 60156,
 'about': 284,
 'school': 67025,
 'life': 44297,
 'such': 74177,
 'teachers': 75997,
 'my': 51490,
 'years': 86260,
 'in': 37733,
 'teaching': 76000,
 'profession': 60088,
 'lead': 43605,
 'me': 47976,
 'to': 77922,
 'believe': 6894,
 'that': 76671,
 'satire': 66462,
 'much': 51018,
 'closer': 14125,
 'reality': 62242,
 'than': 76643,
 'scramble': 67253,
 'survive': 74812,
 'financially': 27904,
 'insightful': 38615,
 'students': 73768,
 'who': 84627,
 'can': 11132,
 'see': 67695,
 'right': 64476,
 'through': 77369,
 'their': 76762,
 'pathetic': 56362,
 'pomp': 58787,
 'pettiness': 57341,
 'of': 53843,
 'whole': 84639,
 'situation': 69959,
 'all': 2038,
 'remind': 63323,
 'schools': 67049,
 'knew': 42226,
 'and': 2762,
 'when': 84450,
 'saw': 66588,

In [60]:
vectorizer_c = CountVectorizer()
vectorizer_c.fit(reviews_train_clean)

# Ya no es binaria la aparicion, sino un conteo por palabra
X_baseline_c = vectorizer_c.transform(reviews_train_clean)

In [61]:
print(X_baseline_c.shape)
print(len(vectorizer_c.get_feature_names())) # Las mismas
# X_baseline_c.toarray()

(25000, 87063)
87063


In [62]:
# Matriz demasiado grande como para que numpy la imprima por pantalla
X_baseline_c

<25000x87063 sparse matrix of type '<class 'numpy.int64'>'
	with 3410713 stored elements in Compressed Sparse Row format>

# Train a Baseline Model

Entrenar un modelo de Regresión Logística después de transformar los datos con CountVectorized

* Son fáciles de interpretar
* Los modelos lineales tienden a funcionar bien en conjuntos de datos dispersos como éste.
* Aprenden muy rápido en comparación con otros algoritmos.

Probar modelos con valores de C de [0.01, 0.05, 0.25, 0.5, 1] y ver cual es el mejor valor para C, y calcular la precisión.

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Los comentarios vienen ordenados. Los primeros 12,5k son positivos
# A test le ocurre lo mismo
target = [1 if i < 12500 else 0 for i in range(25000)]

# target = []
# for i in range(25000):
#     if i < 12500: 
#         target.append(1)
#     else:
#         target.append(0)

def train_model(X_TRAIN, X_TEST):
    
    lr = LogisticRegression()
    
    params = {
        'C': [0.01, 0.05, 0.25, 0.5, 1]
    }
    
    grid = GridSearchCV(lr, params, cv=5)
    grid.fit(X_TRAIN, target)

    print ("Final Accuracy: %s" % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))


In [64]:
train_model(X_baseline, X_test_baseline)

Final Accuracy: 0.88188


# Eliminar las Stop Words

Las stop words son palabras muy comunes como "si", "pero", "nosotros", "él", "ella" y "ellos". Normalmente podemos eliminar estas palabras sin cambiar la semántica de un texto y hacerlo a menudo (aunque no siempre) mejora el rendimiento de un modelo. La eliminación de estas palabras de parada resulta mucho más útil cuando empezamos a utilizar secuencias de palabras más largas como características del modelo (véanse los n-gramas más adelante).

Antes de aplicar el CountVectorized, vamos a eliminar las stopwords, incluidas en nltk.corpus

A continuación, aplica el CountVectorizer y entrena el modelo de regresión logística para obtener la precisión.

In [65]:
# Hay que bajarse las stopwords de nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Fernando
[nltk_data]     Carrasco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
# Para visualizar los stopwords de inglés
from nltk.corpus import stopwords
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [67]:
# Para visualizarlas en español
stopwords.words('spanish')[:10]

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

In [68]:
# Para visualizarlas en español
stopwords.words('chinese')[:10]

['一', '一下', '一些', '一切', '一则', '一天', '一定', '一方面', '一旦', '一时']

In [69]:
stopwords.words('catalan')[:10]

['a',
 'abans',
 'ací',
 'ah',
 'així',
 'això',
 'al',
 'aleshores',
 'algun',
 'alguna']

In [70]:
stopwords.words('basque')[:10]

['ahala',
 'aitzitik',
 'al',
 'ala ',
 'alabadere',
 'alabaina',
 'alabaina',
 'aldiz ',
 'alta',
 'amaitu']

In [71]:
len(stopwords.words('english'))

179

In [72]:
len(stopwords.words('spanish'))

313

In [73]:
from nltk.corpus import stopwords

# Aplicamos la eliminacion de las palabras directamente sobre las reviews
# Demasiado manual. Mejor sobre el CountVectorizer (ver abajo)
english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        
        # Para cada review elimina las stopwords, y separa todas las palabras por espacio
        removed_stop_words.append(
            ' '.join([word for word in review.split() if word not in english_stop_words])
        )
        
    return removed_stop_words

# Se lo aplicamos antes de vectorizar
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [74]:
'''
Vectorizamos tras eliminar las stop words
Ver docu, tiene cosas interesantes como lowercase=True. Lo hace antes de vectorizar, 
o el argumento stopwords
'''
cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)

X = cv.transform(no_stop_words_train)



In [75]:
print(X.shape)

(25000, 87046)


In [76]:
# Se aplica el mismo a test
X_test = cv.transform(no_stop_words_test)

# Y entrenamos
train_model(X, X_test)

Final Accuracy: 0.87936


In [77]:
'''
X_baseline tras aplicar el vectorizador tal cual en los datos
X tras aplicar el vectorizador despues de eliminar las stop words. No se carga muchas
'''
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 87046)
Stop words eliminadas: 17


In [78]:
'''
El resultado de este codigo es practicamente igual que el anterior, pero elimina más stopwords
'''

cv = CountVectorizer(binary=True,
                     stop_words = english_stop_words)

cv.fit(reviews_train_clean)

X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

train_model(X, X_test)

Final Accuracy: 0.87976


In [79]:
'''
X_baseline tras aplicar el vectorizador tal cual en los datos
X tras aplicar el vectorizador despus de eliminar las stop words
En este caso elimina más, el countvectorizer tokeniza mejor las palabras
de lo que lo hemos hecho nosotros en la funcion remove_stop_words. Por ejemplo "it's" serian dos palabras
'''
print(X_baseline.shape)
print(X.shape)
print("Stop words eliminadas:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 86918)
Stop words eliminadas: 145


**Nota:** En la práctica, una manera más fácil de eliminar las stop words es simplemente utilizar el argumento stop_words con cualquiera de las clases 'Vectorizer' de scikit-learn. Si quieres usar la lista completa de stop words de NLTK puedes usar stop_words='english'. En la práctica he encontrado que el uso de la lista de NLTK en realidad disminuye mi rendimiento porque es demasiado amplia, por lo que normalmente proporciono mi propia lista de palabras. Por ejemplo, stop_words=['in','of','at','a','the'] .

Un paso habitual en el preprocesamiento de textos es normalizar las palabras del corpus intentando convertir todas las formas de una palabra en una sola. Para ello existen dos métodos: el Stemming y la Lemmatization.

# Stemming

El "stemming" se considera el método de normalización más tosco o de fuerza bruta (aunque esto no significa necesariamente que vaya a dar peores resultados). Hay varios algoritmos, pero en general todos utilizan reglas básicas para cortar los extremos de las palabras.

NLTK tiene varias implementaciones de algoritmos de stemming. Nosotros usaremos el stemmer Porter. Los más usados:
* PorterStemmer
* SnowballStemmer

Aplicar un PoterStemmer, vectorizar, y entrenar el modelo de nuevo

In [80]:
'''
El stemmer se aplica sobre cada palabra. Las recorta eliminando plurales y tiempos verbales
Modifica muy poco cada palabra
'''

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

plurals = ['caresses', 'flies', 'fly','flight', 'flown', 'dies', 'die', 'mules', 'denied', 'deny',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer', 'colonizing',
            'plotted']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

caress fli fli flight flown die die mule deni deni die agre own humbl size meet state siez item sensat tradit refer colon colon plot


In [81]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

plurals = ['caresses', 'flies', 'fly','flight', 'dies', 'mules', 'denied', 'deny',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

caress fli fli flight die mule deni deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [82]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('spanish')

plurals = ['recorrer', 'corriendo', 'correlación', 'correré', 'casas', 'casero', 'caso', 'playa', 'volando', 'volar', 'volveré']
singles = [stemmer.stem(plural) for plural in plurals]

print(' '.join(singles))

recorr corr correl corr cas caser cas play vol vol volv


In [83]:
# Aplicamos a mano. Los stemmers no eliminan palabras, solo quitan sufijos, y ahora habrá más palabras que sean iguales
from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True, stop_words = english_stop_words)
cv.fit(stemmed_reviews_train)

X_stem = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

In [84]:
train_model(X_stem, X_test)

Final Accuracy: 0.87656


In [85]:
# No elimina palabras. Solo recorta sufijos y agrupa tipos de palabras.
# Como resultado dará menos palabras debido al agrupado. Se carga unas cuantas letras de las palabras
print(X_baseline.shape)
print(X_stem.shape)
print("Diff X normal y X tras stemmer y vectorización:", X_baseline.shape[1] - X_stem.shape[1])

(25000, 87063)
(25000, 66715)
Diff X normal y X tras stemmer y vectorización: 20348


# Lemmatization

La Lemmatization consiste en identificar la parte del discurso de una palabra determinada y, a continuación, aplicar reglas más complejas para transformar la palabra en su verdadera raíz.

In [86]:
import nltk

In [87]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Fernando
[nltk_data]     Carrasco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [88]:
'''
La diferencia con el stemming es que la lematización tiene en cuenta la morfología
de la palabra, sustituyendola por la raiz, no recortándola. Y no es tan restrictivo como el stemming.
Necesita un buen diccionario con mapeos, como wordnet

En nltk no hay lematizadores en español. Habria que bajarse algun paquete como pip install es-lemmatizer
'''

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

plurals = ['caresses', 'flies','fly','flight', 'dies', 'mules', 'studies',
            'died', 'agreed', 'owned', 'humbled', 'sized',
            'meeting', 'stating', 'siezing', 'itemization',
            'sensational', 'traditional', 'reference', 'colonizer',
            'plotted']
singles = [lemmatizer.lemmatize(plural) for plural in plurals]

print(' '.join(singles))

caress fly fly flight dy mule study died agreed owned humbled sized meeting stating siezing itemization sensational traditional reference colonizer plotted


In [89]:
def get_lemmatized_text(corpus):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

# Lematizamos las reviews
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

# Vectorizamos con conteo tras lematizar
cv = CountVectorizer(binary=True, stop_words = english_stop_words)
cv.fit(lemmatized_reviews_train)

X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

train_model(X, X_test)

Final Accuracy: 0.87812


In [90]:
# Elimina menos que con el stemmer. Normal, el stemmer recorta mucho del sufijo
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 80181)
Diff X normal y X tras lematizador y vectorización: 6882


# N-grams

Podemos añadir potencialmente más poder predictivo a nuestro modelo añadiendo también secuencias de dos o tres palabras (bigramas o trigramas). Por ejemplo, si una crítica tuviera la secuencia de tres palabras "no me gustó la película", sólo consideraríamos estas palabras individualmente con un modelo de unigramas y probablemente no captaríamos que se trata de un sentimiento negativo, porque la palabra "me gustó" por sí sola va a estar muy correlacionada con una crítica positiva.

La biblioteca scikit-learn hace que sea muy fácil jugar con esto. Sólo tiene que utilizar el argumento ngram_range con cualquiera de las clases 'Vectorizer'.

In [91]:
from nltk import ngrams

sentence = "didn't love music at all my love"

one = ngrams(sentence.split(), 1)
two = ngrams(sentence.split(), 2)
three = ngrams(sentence.split(), 3)

for grams in one:
  print(grams)

for grams in two:
  print(grams)
print('###############')
for grams in three:
  print(grams)



("didn't",)
('love',)
('music',)
('at',)
('all',)
('my',)
('love',)
("didn't", 'love')
('love', 'music')
('music', 'at')
('at', 'all')
('all', 'my')
('my', 'love')
###############
("didn't", 'love', 'music')
('love', 'music', 'at')
('music', 'at', 'all')
('at', 'all', 'my')
('all', 'my', 'love')


In [1]:
'''
Puede ser bigramas si ngram_range=(2,2), o trigramas (3,3)...
Algunas palabras las elimina, como 'a'. Cuidado con eso a la hora de hacer el conteo
ngram_range=(1, 3) significa las palabras por separado, los bigramas y los trigramas
Ojo que esto aumenta muchisimo el espacio de features
'''
ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 2))

vector = ngram_vectorizer.fit_transform([sentence]).toarray()
print(vector)
print(len(vector[0])+1)

NameError: name 'CountVectorizer' is not defined

In [93]:
'''
Va como argumento del CountVectorizer
'''
ngram_vectorizer = CountVectorizer(binary=True, stop_words = english_stop_words,
                                   ngram_range=(1, 2))

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model(X, X_test)

Final Accuracy: 0.889


In [94]:
# Añade 1448047 n gramas. Cuanto mas ngramas, mayor será el espacio de features
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 1865232)
Diff X normal y X tras lematizador y vectorización: -1778169


In [95]:
'''
Va como argumento del CountVectorizer
'''
ngram_vectorizer = CountVectorizer(binary=True, stop_words = english_stop_words,
                                   ngram_range=(2, 2))

ngram_vectorizer.fit(reviews_train_clean)

X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

train_model(X, X_test)
# Esto da 0.84056, como podemos ver la predicción por bigramas es menos potente

(25000, 87063)
(25000, 1778314)
Diff X normal y X tras lematizador y vectorización: -1691251


# TF-IDF

Otra forma habitual de representar cada documento de un corpus es utilizar el estadístico tf-idf (frecuencia de términos-frecuencia inversa de documentos) para cada palabra, que es un factor de ponderación que podemos utilizar en lugar de las representaciones binarias o de recuento de palabras.

Hay varias formas de realizar la transformación tf-idf, pero en pocas palabras, **tf-idf pretende representar el número de veces que una palabra dada aparece en un documento (una crítica de cine en nuestro caso) en relación con el número de documentos del corpus en los que aparece la palabra**.

**Nota: Ahora que ya hemos hablado de los n-gramas, cuando hablo de "palabras" me refiero a cualquier n-grama (secuencia de palabras) si el modelo utiliza un n mayor que uno.

In [96]:
'''
ln(N + 1 / count + 1) + 1
Cuanto más común, menor es el TDFIDF. Cuanto más rara, mayor es el valor
'''
# Numero de documentos
N = 3

# Numero de veces que aparece
count = 2

1 + np.log((N + 1)/(count + 1))

1.2876820724517808

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfTransformer
'''
Cuanto más común, más bajo es el TfidfVectorizer
'''
sent1 = 'My name is Ralph'
sent2 = 'Ralph is fat'
sent3 = 'Ralph'

test = TfidfVectorizer()
test.fit_transform([sent1, sent2, sent3])
print(test.idf_)
print(test.get_feature_names())

[1.69314718 1.28768207 1.69314718 1.69314718 1.        ]
['fat', 'is', 'my', 'name', 'ralph']


In [2]:
# 1 + ln(N + 1 / count + 1)
1 + np.log((3 + 1)/(1 + 1))

NameError: name 'np' is not defined

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

train_model(X, X_test)

Final Accuracy: 0.8822


In [100]:
print(X_baseline.shape)
print(X.shape)
print("Diff X normal y X tras lematizador y vectorización:", X_baseline.shape[1] - X.shape[1])

(25000, 87063)
(25000, 87063)
Diff X normal y X tras lematizador y vectorización: 0


# Support Vector Machines (SVM)

Recordemos que los clasificadores lineales tienden a funcionar bien en conjuntos de datos muy dispersos (como el que tenemos). Otro algoritmo que puede producir grandes resultados con un tiempo de entrenamiento rápido son las máquinas de vectores de soporte con un núcleo lineal.

Construya un modelo con un rango de n-gramas de 1 a 2:

In [101]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# SVM con bigramas
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)



def train_model_svm(X_TRAIN, X_TEST):
    
    svm = LinearSVC()
    
    params = {
        'C': [0.01, 0.05, 0.25, 0.5, 1]
    }
    
    grid = GridSearchCV(svm, params, cv=5)
    grid.fit(X_TRAIN, target)

    print ("Final Accuracy: %s" % accuracy_score(target, grid.best_estimator_.predict(X_TEST)))
    

train_model_svm(X, X_test)

Final Accuracy: 0.89768


# Modelo final

La eliminación de un pequeño conjunto de palabras vacías junto con un rango de n-gramas de 1 a 3 y un clasificador lineal de vectores de soporte muestra los mejores resultados.

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC


stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True,
                                   ngram_range=(1, 2),
                                   stop_words=stop_words)

ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

train_model_svm(X, X_test)

Final Accuracy: 0.89924


# Principales features positivas y negativas

Obtener las características más importantes del modelo.

In [105]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)

log_reg = LogisticRegression(C=0.5)
log_reg.fit(X, target)

# Importancia de los coeficientes. En total, todas las palabras vectorizadas
print(len(log_reg.coef_[0]))

# Cada coeficiente va asociado a una palabra
cv.get_feature_names()

# Montamos un diccionario con palabra -> coeficiente
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), log_reg.coef_[0]
    )
}

87063


In [106]:
log_reg.predict(cv.transform(['This movie is horrible']))

array([0])

In [ ]:
log_reg.predict(cv.transform(['This movie is incredible']))

In [107]:

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print(best_positive)
    
print('################################')
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print(best_negative)
    

('excellent', 1.3792554670979043)
('refreshing', 1.2812203080059015)
('perfect', 1.200895573911889)
('superb', 1.1359361912434265)
('appreciated', 1.1341593639571255)
################################
('worst', -2.0614270242430086)
('waste', -1.9173223247656896)
('disappointment', -1.676682059289742)
('poorly', -1.6573838956395175)
('awful', -1.5339854389777068)
